In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, ElementNotInteractableException, ElementClickInterceptedException
from bs4 import BeautifulSoup
from selenium.webdriver.chrome.options import Options
import time
import pandas as pd
from datetime import datetime
from hdfs import InsecureClient


In [2]:
print("a")

a


In [3]:
def get_country():
    chrome_options = Options()
    chrome_options.add_argument("--headless")
    driver = webdriver.Chrome(options = chrome_options)
    try:
        url = "https://www.flightradar24.com/data/airports"
        driver.get(url)

        # Find all links to countries
        country_links = driver.find_elements(By.CSS_SELECTOR, 'table#tbl-datatable a[href^="https://www.flightradar24.com/data/airports/"]')
        country = []
        # Extract and print the country names
        for country_link in country_links:
            country_name = country_link.get_attribute("title")
            country.append(country_name.replace(" ", "-").replace("(", "").replace(")", ""))
        country_set = set(country)
    finally:
        driver.quit()
        return (country_set)

In [4]:

def scrape_airport_data(country_url):
    chrome_options = Options()
    chrome_options.add_argument("--headless")
    driver = webdriver.Chrome(options = chrome_options)

    try:
        driver.get(country_url)

        # Find all airport links
        IATA = []
        airport_names = []
        #k = 0
        airport_links = driver.find_elements(By.CSS_SELECTOR, 'a[data-iata][data-lat][data-lon]')
        for airport_link in airport_links:
            airport_name = airport_link.text.strip().split('\n')[0]
            airport_iata = airport_link.get_attribute('data-iata')
            airport_icao = airport_name.split('(')[-1].split(')')[0]
            IATA.append(airport_iata)
            airport_names.append(airport_name)
        airport_names_cleaned = [airport.split(' (')[0] for airport in airport_names]
 
    finally:
        driver.quit()
    return IATA , airport_names_cleaned


In [5]:
def scrape_weather_data (airport_code) :
    chrome_options = Options()
    chrome_options.add_argument("--headless")
    driver = webdriver.Chrome(options = chrome_options)
    
    airport_url = "https://www.flightradar24.com/data/airports/"+airport_code+"/weather"
    try:
        driver.get(airport_url)

        # Wait for the button to appear and accept cookies
        time.sleep(15)
        try:
            button1 = driver.find_element(By.ID, 'onetrust-accept-btn-handler')
            button1.click()
        except NoSuchElementException:
            pass  # If the accept cookies button is not found, continue without clicking
        
        try:
            # Find the element with the specified class name
            alert_element = driver.find_element(By.CSS_SELECTOR, 'div.alert.alert-info.text-center')
        except:
            alert_element = "No alert" 
        if alert_element == "No alert" :
            element = driver.find_elements(By.CSS_SELECTOR,'td[colspan="2"] > ul')
            weather_data = []
            for e in element:
                soup = e.get_attribute('outerHTML')
                sp = BeautifulSoup(soup, 'html.parser')
                
                ul_tag = sp.find('ul')
                # Extract all the information
                information = {}
                for li_tag in ul_tag.find_all('li'):
                    key = li_tag.find('strong').text.strip().rstrip(':')
                    value = li_tag.text.strip().replace(key + ':', '').strip()
                    information[key] = value

            
                # Append data to the list as a dictionary
                weather_data.append(information)
            df = pd.DataFrame(weather_data)
            # Find all elements with the specified classes
            elements = driver.find_elements(By.XPATH , '//tr[@class="master expandable"]')
            # Extract and print the date from each element
            dates = []
            for element in elements:
                soup = element.get_attribute('outerHTML')
                sp = BeautifulSoup(soup, 'html.parser')
                tr_element = sp.find('tr', class_='master expandable')
                # Extract the text content of the second <td> element within the <tr> element
                date_td = tr_element.find_all('td')[1]  # Second <td> element contains the date
                date = date_td.text.strip()
                
                #print("Date:", date)
                dates.append(date)
            df["date"] = dates
            #print (dates, len(dates))
            return (df)
        else : 
          
            return "we don't have any data for this ariport" 


    finally:
            driver.quit()


In [6]:
scrape_weather_data("tun")

,Day,Time,Wind direction,Wind speed,Temperature,Dew point,Pressure,Visibility,Wind gust,date
0,18th,08:00 UTC,170°,5kt,16°C,11°C,1020 hPa,Sky and visibility OK,NaN,2024-03-18 08:00:00
1,18th,07:30 UTC,180°,7kt,15°C,11°C,1020 hPa,8000 m,NaN,2024-03-18 07:30:00
2,18th,07:00 UTC,180°,6kt,13°C,11°C,1020 hPa,8000 m,NaN,2024-03-18 07:00:00
3,18th,06:30 UTC,170°,4kt,12°C,11°C,1020 hPa,8000 m,NaN,2024-03-18 06:30:00
4,18th,06:00 UTC,170°,4kt,11°C,10°C,1020 hPa,8000 m,NaN,2024-03-18 06:00:00
...,...,...,...,...,...,...,...,...,...,...
135,15th,11:00 UTC,300°,3kt,19°C,10°C,1021 hPa,9999 m,NaN,2024-03-15 11:00:00
136,15th,10:30 UTC,NaN,2kt,19°C,12°C,1021 hPa,9999 m,NaN,2024-03-15 10:30:00
137,15th,10:00 UTC,NaN,2kt,19°C,12°C,1021 hPa,9999 m,NaN,2024-03-15 10:00:00
138,15th,09:30 UTC,260°,3kt,18°C,11°C,1021 hPa,9999 m,NaN,2024-03-15 09:30:00


# -------------------------

In [7]:
def scrape_weather_from_country (country_name):
    url_country = "https://www.flightradar24.com/data/airports/" + country_name
    IATA , aeroport_names = scrape_airport_data(url_country)
    data_total_country = []
    for i, j in  zip (IATA, aeroport_names) : 
        df_aeroport = scrape_weather_data(i) 
        if (type(df_aeroport) != str):
            print("data founded in ",j , " = " , len(df_aeroport)  )
            df_aeroport["Original Aeroport"] =  j 
            data_total_country.append(df_aeroport)
    if len(data_total_country) >0 :
        concatenated_df = pd.concat(data_total_country, ignore_index=True)
        if isinstance(concatenated_df, pd.DataFrame) :
            return concatenated_df
    else :
        return "No data"


In [8]:
scrape_weather_from_country("tunisia")

data founded in  Djerba Zarzis International Airport  =  140
data founded in  Enfidha Hammamet International Airport  =  141
data founded in  Gabes Matmata International Airport  =  72
data founded in  Monastir Habib Bourguiba International Airport  =  142
data founded in  Sfax–Thyna International Airport  =  70
data founded in  Tabarka Ain Draham Airport  =  69
data founded in  Tozeur Nefta International Airport  =  71
data founded in  Tunis Carthage International Airport  =  139


,Day,Time,Wind direction,Wind speed,Temperature,Dew point,Pressure,Visibility,date,Original Aeroport,Wind gust
0,18th,08:00 UTC,170°,5kt,18°C,10°C,1022 hPa,Sky and visibility OK,2024-03-18 08:00:00,Djerba Zarzis International Airport,NaN
1,18th,07:30 UTC,160°,2kt,17°C,12°C,1022 hPa,Sky and visibility OK,2024-03-18 07:30:00,Djerba Zarzis International Airport,NaN
2,18th,07:00 UTC,150°,2kt,15°C,12°C,1022 hPa,Sky and visibility OK,2024-03-18 07:00:00,Djerba Zarzis International Airport,NaN
3,18th,06:30 UTC,160°,1kt,13°C,12°C,1022 hPa,Sky and visibility OK,2024-03-18 06:30:00,Djerba Zarzis International Airport,NaN
4,18th,06:00 UTC,170°,2kt,12°C,11°C,1022 hPa,Sky and visibility OK,2024-03-18 06:00:00,Djerba Zarzis International Airport,NaN
...,...,...,...,...,...,...,...,...,...,...,...
839,15th,11:30 UTC,NaN,2kt,19°C,11°C,1021 hPa,9999 m,2024-03-15 11:30:00,Tunis Carthage International Airport,NaN
840,15th,11:00 UTC,300°,3kt,19°C,10°C,1021 hPa,9999 m,2024-03-15 11:00:00,Tunis Carthage International Airport,NaN
841,15th,10:30 UTC,NaN,2kt,19°C,12°C,1021 hPa,9999 m,2024-03-15 10:30:00,Tunis Carthage International Airport,NaN
842,15th,10:00 UTC,NaN,2kt,19°C,12°C,1021 hPa,9999 m,2024-03-15 10:00:00,Tunis Carthage International Airport,NaN


In [9]:
def send_to_hdfs(data, file_path): 
    client = InsecureClient('http://localhost:50070')
    with client.write(file_path, overwrite=True) as writer:
        data.to_csv(writer, index=False)


In [10]:
import os
import csv

def save_csv(data, parent_folder, folder_name, file_name):
    # Create the folder if it doesn't exist
    folder_path = os.path.join(parent_folder, folder_name)
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)

    # Define the file path within the new folder
    file_path = os.path.join(folder_path, file_name)

    send_to_hdfs(data , file_path.replace(os.path.sep, '/'))



In [11]:
from IPython.display import HTML

def scrape_all_arrival_flight() :
    countries = get_country()
  
    #countries =["Egypt","Saudi-Arabia","United Arab Emirates" ,"Algeria","Morocco","Tunisia"]
   # countries_done = ["Italy","Somalia","Serbia","Botswana","Aruba","Nigeria","Antigua-And-Barbuda","Democratic-Republic-Of-The-Congo","Czechia","Zimbabwe","Venezuela","Slovakia","Uzbekistan","Equatorial-Guinea","Virgin-Islands-Us","Greenland","Iraq","Argentina","El-Salvador",'Sierra-Leone',"Chile","Marshall-Islands", "Belarus","Barbados","Suriname","Saint-Helena","Martinique","Ukraine","Egypt", "Russia","Martinique","Moldova","Ethiopia" , "Dominican-Republic", "Austria","Ukraine","New-Caledonia", "Senegal","Poland" , "Puerto-Rico","Slovenia" , "Micronesia" , "Mauritius","Turkmenistan","Zambia","tunisia" , "Solomon-Islands" ,"Cocos-keeling-Islands" ]
    for country in countries :
       # if country not in countries_done:
            print("start of extract data from :", country)
            data = scrape_weather_from_country(country) 
            print("end of extract data from :", country)
           # countries_done.append(country)
            # File path to save the CSV data
            
            date = datetime.now().date()
            # File path to save the CSV data
            file_name = country + str(date) +".csv"
            # Save the DataFrame to a CSV file
            parent_folder = "/user/PFE_data/weather_flights"
            #folder name 
            folder_name = country

            if isinstance(data, pd.DataFrame) :
                save_csv(data,parent_folder ,folder_name,file_name )
                message = '<p style="color:green;">data saved</p>'
                display(HTML(message))


In [ ]:
scrape_all_arrival_flight()

start of extract data from : French-Guiana
data founded in  Cayenne Felix Eboue Airport  =  144
end of extract data from : French-Guiana


start of extract data from : Czechia
data founded in  Brno Turany Airport  =  143
data founded in  Karlovy Vary Airport  =  111
data founded in  Kunovice Airport  =  28
data founded in  Ostrava Leos Janacek Airport  =  0
data founded in  Pardubice Airport  =  71
data founded in  Plzen Line Airport  =  71
data founded in  Prague Vaclav Havel Airport  =  144
data founded in  Prague Kbely Airport  =  70
data founded in  Vodochody Airport  =  5
end of extract data from : Czechia


start of extract data from : Faroe-Islands
data founded in  Vagar Airport  =  144
end of extract data from : Faroe-Islands


start of extract data from : Solomon-Islands
